In [1]:
bucket = 'arch-ml-bucket-us-east-1'
prefix = 'mall_customer'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
import os
import pandas as pd
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker
from sagemaker.predictor import csv_serializer, json_deserializer
import matplotlib.pyplot as plt
import mxnet as mx
import numpy as np
from scipy.spatial.distance import cdist
%matplotlib inline

In [2]:
role = get_execution_role()

# Data Preprocessing

In [13]:
df = pd.read_csv('Mall_Customers.csv')
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [14]:
df.shape

(200, 5)

In [15]:
features = ['Annual Income (k$)', 'Spending Score (1-100)']
train_df = df[features]

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df = scaler.fit_transform(train_df)
train_df = pd.DataFrame(data=train_df, columns=features)
train_df = train_df.astype('float32')

In [8]:
train_df.to_csv('train.csv', header=False, index=False)

In [9]:
# Write Column List

with open('feature_list.txt','w') as f:
    f.write(','.join(features))

In [18]:
import pickle

with open('scaler.pkl', 'wb') as f:
    pickle.dump(obj=scaler, file=f)    

# Upload data to S3

In [10]:
def write_to_s3(filename):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket)\
            .Object(os.path.join(prefix, filename)).upload_fileobj(f)

In [11]:
write_to_s3('train.csv')